In [15]:
#@title setup

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa


from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from functools import partial
from IPython.display import HTML, IFrame
from jaxtyping import Float
from os import environ
from transformer_lens import ActivationCache, HookedTransformer
from typing import List, Optional, Union
import circuitsvis as cv
import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import tqdm.auto as tqdm
import transformer_lens
import transformer_lens.utils as utils
import itertools


In [ ]:
#@title Load Model

# plotly.offline.init_notebook_mode()

torch.set_grad_enabled(False)

DEVICE = utils.get_device()
print(f'{DEVICE=}')

PRETRAINED_MODEL = 'gpt2'  # gpt2-small or gpt2-medium
print(f'{PRETRAINED_MODEL=}')

model = transformer_lens.HookedTransformer.from_pretrained(
    PRETRAINED_MODEL, center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True, device=DEVICE)

print("print model structure", model)



DEVICE=device(type='mps')
PRETRAINED_MODEL='gpt2'
Loaded pretrained model gpt2 into HookedTransformer
print model structure HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input)

100%|██████████| 10/10 [00:00<00:00, 10.15it/s]


123 + 456 = 12774 Business use per year and 10%: Service


100%|██████████| 10/10 [00:00<00:00, 22.35it/s]


The Big Bang Theory premieres on ABC Wednesday, March 26 and airs Sunday evenings at


100%|██████████| 10/10 [00:00<00:00, 25.50it/s]

Beats Music is owned by Carl Peterson. Its popularity so far inspires skepticism by


In [ ]:
print(model.generate("123 + 456 ="))
print(model.generate("The Big Bang Theory premieres on"))
print(model.generate("Beats Music is owned by"))
print(model.generate("Michael Jordan plays the sport of"))
utils.test_prompt(prompt='Michael Jordan plays the sport of', answer=' basketball', model=model, prepend_space_to_answer=False)


100%|██████████| 10/10 [00:00<00:00, 10.99it/s]


123 + 456 = n2_pickle 4 + 1 Amazing mis


100%|██████████| 10/10 [00:00<00:00, 21.92it/s]


The Big Bang Theory premieres on PBS Tuesdays at 8/7c and hits


100%|██████████| 10/10 [00:00<00:00, 25.05it/s]


...

Michael Jordan plays the sport of Jamaican hockey on ice at the away, Division
Tokenized prompt: ['<|endoftext|>', 'Michael', ' Jordan', ' plays', ' the', ' sport', ' of']
Tokenized answer: [' basketball']


Performance on answer token:
Rank: 0        Logit: 16.17 Prob: 50.01% Token: | basketball|

Top 0th token. Logit: 16.17 Prob: 50.01% Token: | basketball|
Top 1th token. Logit: 13.77 Prob:  4.53% Token: | football|
Top 2th token. Logit: 13.72 Prob:  4.30% Token: | golf|
Top 3th token. Logit: 13.08 Prob:  2.27% Token: | tennis|
Top 4th token. Logit: 12.90 Prob:  1.90% Token: | soccer|
Top 5th token. Logit: 12.89 Prob:  1.87% Token: | hockey|
Top 6th token. Logit: 12.55 Prob:  1.33% Token: | boxing|
Top 7th token. Logit: 12.38 Prob:  1.13% Token: | sports|
Top 8th token. Logit: 12.24 Prob:  0.98% Token: | the|
Top 9th token. Logit: 12.16 Prob:  0.90% Token: | baseball|


Ranks of the answer tokens: [(' basketball', 0)]

tensor([50256,   642], device='mps:0')


In [ ]:
#@title Experiment on tokenization

prompt = "123+456="

# for i in range(len(prompt)):
# add space in various places in the prompt
for add_space_indices in itertools.product([True, False], repeat=len(prompt)):
  prompt_with_space = prompt
  for i, bool_add_space in zip(reversed(range(len(prompt))), reversed(add_space_indices)):
    if bool_add_space:
      prompt_with_space = prompt_with_space[:i] + " " + prompt_with_space[i:]
  print(f'\n{prompt_with_space=}')
  # utils.test_prompt(prompt=prompt_with_space, answer=' 579', model=model, top_k=1)
  utils.test_prompt(prompt=prompt_with_space, answer='579', model=model, top_k=1, prepend_space_to_answer=False)
  print("\n")
  utils.test_prompt(prompt=prompt_with_space, answer=' 579', model=model, top_k=1, prepend_space_to_answer=False)



# for p in prompts:
#   print(f"\nprompt=|{p}|, answer=| 579|")
#


top results:

```


Tokenized prompt: ['<|endoftext|>', ' 1', ' 2', ' 3', ' +', ' 45', ' 6', '=']
Tokenized answer: [' 5', '79']
Performance on answer token:
Rank: 2        Logit: 11.92 Prob:  1.28% Token: | 5|
Top 0th token. Logit: 12.34 Prob:  1.94% Token: | 1|
Performance on answer token:
Rank: 309      Logit:  6.43 Prob:  0.03% Token: |79|
Top 0th token. Logit: 12.09 Prob:  9.01% Token: |
|
Ranks of the answer tokens: [(' 5', 2), ('79', 309)]

Tokenized prompt: ['<|endoftext|>', ' 12', ' 3', ' +', ' 4', ' 5', ' 6', '=']
Tokenized answer: [' 5', '79']
Performance on answer token:
Rank: 3        Logit: 11.82 Prob:  2.44% Token: | 5|
Top 0th token. Logit: 12.56 Prob:  5.13% Token: |
|
Performance on answer token:
Rank: 1747     Logit:  4.54 Prob:  0.00% Token: |79|
Top 0th token. Logit: 12.84 Prob:  9.16% Token: |
|
Ranks of the answer tokens: [(' 5', 3), ('79', 1747)]

Tokenized prompt: ['<|endoftext|>', '1', ' 2', ' 3', ' +', ' 45', ' 6', '=']
Tokenized answer: [' 5', '79']
Performance on answer token:
Rank: 3        Logit: 11.91 Prob:  1.23% Token: | 5|
Top 0th token. Logit: 12.24 Prob:  1.71% Token: | 1|
Performance on answer token:
Rank: 283      Logit:  6.63 Prob:  0.04% Token: |79|
Top 0th token. Logit: 12.05 Prob:  8.38% Token: |
|
Ranks of the answer tokens: [(' 5', 3), ('79', 283)]

```



In [32]:
# @title acceptable token on the answers "579" and some wrong answer tokens

# ['<|endoftext|>', ' 1', ' 2', ' 3', ' +', ' 45', ' 6', '=']
# input_token = " 1 2 3 + 45 6="
# answer_token = " 579"
# right_token_strs = ['5', '57', '579', ' 5', ' 57']
# wrong_token_strs = ['1', ' 1', '12', ' 7']


# ===========
"""
Tokenized prompt: ['<|endoftext|>', 'Michael', ' Jordan', ' plays', ' the', ' sport', ' of']
Tokenized answer: [' basketball']
Performance on answer token:
Rank: 0        Logit: 16.17 Prob: 50.01% Token: | basketball|
Top 0th token. Logit: 16.17 Prob: 50.01% Token: | basketball|
Top 1th token. Logit: 13.77 Prob:  4.53% Token: | football|
Top 2th token. Logit: 13.72 Prob:  4.30% Token: | golf|
Top 3th token. Logit: 13.08 Prob:  2.27% Token: | tennis|
Top 4th token. Logit: 12.90 Prob:  1.90% Token: | soccer|
Top 5th token. Logit: 12.89 Prob:  1.87% Token: | hockey|
Top 6th token. Logit: 12.55 Prob:  1.33% Token: | boxing|
Top 7th token. Logit: 12.38 Prob:  1.13% Token: | sports|
Top 8th token. Logit: 12.24 Prob:  0.98% Token: | the|
Top 9th token. Logit: 12.16 Prob:  0.90% Token: | baseball|
Ranks of the answer tokens: [(' basketball', 0)]
"""
input_token = "Michael Jordan plays the sport of"
answer_token = " basketball"
right_token_strs = [' basketball']
wrong_token_strs = [' football', 'golf']
# ===========

output_logit, cache = model.run_with_cache(input_token)
print(f'{output_logit.shape=}')

right_tokens = []
wrong_tokens = []
for out_token_array, token_strs in [(right_tokens, right_token_strs), (wrong_tokens, wrong_token_strs)]:
    for token_str in token_strs:
        print(
            f'|{token_str}| \t token: {model.to_tokens(token_str, prepend_bos=False)[0][0]} \t logit: {output_logit[0, -1, model.to_tokens(token_str, prepend_bos=False)[0]]}')
        out_token_array.append(model.to_tokens(token_str, prepend_bos=False)[0][0])

accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True, apply_ln=True
)
accumulated_residual_logits = accumulated_residual @ model.unembed.W_U + model.unembed.b_U
fig = go.Figure()
for token_str, t in zip(right_token_strs, right_tokens):
    fig.add_trace(go.Scatter(
        x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
        y=utils.to_numpy(accumulated_residual_logits[:, 0, t]),
        text=labels,
        name=token_str,
        # line=dict(
        #     color='blue',
        #     # dash ='dot',
        #     # width=3.5,
        # )
    ))
for token_str, t in zip(wrong_token_strs, wrong_tokens):
    fig.add_trace(go.Scatter(
        x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
        y=utils.to_numpy(accumulated_residual_logits[:, 0, t]),
        text=labels,
        name=token_str,
        # color='royalblue',
        line=dict(
            # color='red',
            dash='dot',
            # width=3.5,
        )
    ))
fig.show()

output_logit.shape=torch.Size([1, 7, 50257])
| basketball| 	 token: 9669 	 logit: tensor([16.1726], device='mps:0')
| football| 	 token: 4346 	 logit: tensor([13.7722], device='mps:0')
|golf| 	 token: 70 	 logit: tensor([2.7025e+00, 1.0197e-03], device='mps:0')


In [36]:
stack_head_result, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True, apply_ln=True
)
stack_head_result = stack_head_result @ model.unembed.W_U + model.unembed.b_U
stack_head_result = stack_head_result.view(12, 12, -1)

figs = make_subplots(rows=1, cols=len(right_token_strs), subplot_titles=right_token_strs)
i = 1
for token_str, t in zip(right_token_strs, right_tokens):
  subfig = px.imshow(
      utils.to_numpy(stack_head_result[:,:, t]),
      labels={"x": "Head", "y": "Layer"},
  )
  for trace in subfig.data:
    figs.add_trace(trace, row=1, col=i)
  i = i + 1
figs.show()

figs = make_subplots(rows=1, cols=len(wrong_token_strs), subplot_titles=wrong_token_strs)
i = 1
for token_str, t in zip(wrong_token_strs, wrong_tokens):
  subfig = px.imshow(
      utils.to_numpy(stack_head_result[:,:, t]),
      labels={"x": "Head", "y": "Layer"},
  )
  for trace in subfig.data:
    figs.add_trace(trace, row=1, col=i)
  i = i + 1
figs.show()




In [ ]:
clean_prompt = ""
corrupted_prompt = "After John and Mary went to the store, John gave a bottle of milk to"

clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

def residual_stream_patching_hook(resid_pre, hook, position):
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, position, :]
    return resid_pre


# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
print(f'{clean_tokens.shape=}')
num_positions = len(clean_tokens[0])
print(f'{num_positions=}')
ioi_patching_result = torch.zeros(
    (model.cfg.n_layers, num_positions), device=model.cfg.device)

for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(
            residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = model.run_with_hooks(corrupted_tokens, fwd_hooks=[
            (utils.get_act_name("resid_pre", layer), temp_hook_fn)
        ])
        # Calculate the logit difference
        patched_logit_diff = logits_to_logit_diff(patched_logits, correct_answer=" Seattle", incorrect_answer=" Paris").detach()
        # Store the result, normalizing by the clean and corrupted logit difference so it's between 0 and 1 (ish)
        ioi_patching_result[layer, position] = (
            patched_logit_diff - corrupted_logit_diff)/(clean_logit_diff - corrupted_logit_diff)

token_labels = [f"{token}_{index}" for index, token in enumerate(model.to_str_tokens(clean_tokens))]
imshow(ioi_patching_result, x=token_labels, xaxis="Position", yaxis="Layer", title="Normalized Logit Difference After Patching Residual Stream on the IOI Task")